In [ ]:
!docker run --name mongodb -d -p 27017:27017 mongo:latest


In [ ]:
!docker ps


In [ ]:
%pip install llama-index-storage-docstore-mongodb
%pip install llama-index-storage-index-store-mongodb
%pip install llama-index


In [ ]:
import os
import logging
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex, SimpleKeywordTableIndex, SummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.storage.docstore.mongodb import MongoDocumentStore
from llama_index.storage.index_store.mongodb import MongoIndexStore


In [ ]:

# Configure logging
logging.basicConfig(level=logging.INFO)

# MongoDB connection URI
MONGO_URI = "mongodb://localhost:27017"


In [ ]:
#os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")
from llama_index.core import Settings
from llama_index.llms.ollama import  Ollama
Settings.llm = Ollama(model='llama3.2:latest', base_url='http://localhost:11434',temperature=0.1)



In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",  # Replace with your desired model
    base_url="http://localhost:11434",  # Ensure Ollama is running at this endpoint
    ollama_additional_kwargs={"mirostat": 0} #Mirostat is a technique for controlling perplexity and balancing the text generation process in large language models (LLMs).
) 

In [ ]:
import json
# Load the saved index IDs
with open("mongodb_index_ids.json", "r") as f:
    index_ids = json.load(f)

summary_id = index_ids["summary_id"]
vector_id = index_ids["vector_id"]
keyword_id = index_ids["keyword_id"]

# Re-create storage context
storage_context = StorageContext.from_defaults(
    docstore=MongoDocumentStore.from_uri(uri=MONGO_URI),
    index_store=MongoIndexStore.from_uri(uri=MONGO_URI),
)

# Reload indices
summary_index = load_index_from_storage(storage_context=storage_context, index_id=summary_id)
vector_index = load_index_from_storage(storage_context=storage_context, index_id=vector_id, embed_model=ollama_embedding)
keyword_table_index = load_index_from_storage(storage_context=storage_context, index_id=keyword_id)

print("Indices reloaded successfully.")

In [ ]:
nodes = summary_index.storage_context.docstore.docs.values()
for node in nodes:
    print(f"Node Text: {node.text}")
    print(f"Node Metadata: {node.extra_info}")

In [ ]:
# Query Summary Index
query_engine = summary_index.as_query_engine()
summary_response = query_engine.query("What is a summary of this document?")
print(summary_response)

In [ ]:
# Query Summary Index
query_engine = summary_index.as_query_engine()
summary_response = query_engine.query("What is a summary of this document?")
print(summary_response)

# Query Vector Index
query_engine = vector_index.as_query_engine()
vector_response = query_engine.query("What did the author do growing up?")
print(vector_response)

# Query Keyword Table Index
query_engine = keyword_table_index.as_query_engine()
keyword_response = query_engine.query("What did the author do after his time at YC?")
print(keyword_response)


In [34]:
from llama_index.core.response.notebook_utils import display_response

display_response(summary_response)
display_response(vector_response)
display_response(keyword_response)

**`Final Response:`** The document appears to be a collection of anecdotes and reflections from the writer's experiences in the startup world, particularly at Viaweb and Y Combinator. The writer shares stories about the challenges they faced, such as navigating the complexities of online publishing, dealing with criticism for their technical choices, and adjusting to new realities after being acquired by Yahoo.

The writer also discusses how these experiences made them realize that they needed to make a change and eventually left Y Combinator. Overall, the document provides a glimpse into the writer's personal struggles and growth as an entrepreneur, highlighting the challenges and lessons learned from their time in the startup world.

**`Final Response:`** Empty Response

**`Final Response:`** The author shifted his focus to other pursuits after leaving YC. He began exploring painting as a new creative outlet, allowing him to dedicate himself fully to it. This endeavor provided an opportunity for personal growth and self-discovery. Although he initially showed promise, the author eventually lost interest in painting and redirected his attention towards writing essays once more. Additionally, he delved into Lisp programming, a language that had long fascinated him.

**`Final Response:`** The author shifted his focus to other pursuits after leaving YC. He began exploring painting as a new creative outlet, allowing him to dedicate himself fully to it. This endeavor provided an opportunity for personal growth and self-discovery. Although he initially showed promise, the author eventually lost interest in painting and redirected his attention towards writing essays once more. Additionally, he delved into Lisp programming, a language that had long fascinated him.